# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
tf_df = pd.read_csv("../output_data/cities.csv")

tf_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity & locations to float and store and drop empty rows
tf_drop = tf_df.dropna()
humidity = tf_drop["Humidity"].astype(float) 
locations =tf_drop[["Lat", "Lng"]].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(tf_drop["Humidity"]),
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 70 to 444
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Lat     7 non-null      float64
 1   Lng     7 non-null      float64
dtypes: float64(2)
memory usage: 488.0 bytes


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# A max temperature between 70  and  80 degrees.
weather_df = tf_df.loc[(tf_df["Max Temp"] < 80) & (tf_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
weather_df_less_10 =weather_df.loc[weather_df["Wind Speed"] < 10]

# for cloudiness equals to  Zero .
weather_df_equal_0 = weather_df_less_10.loc[weather_df_less_10["Cloudiness"] == 0]

# Drop any rows with null values
fit_weather = weather_df_equal_0.dropna()

fit_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = fit_weather

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # update location key value 
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
   
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except(KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    



Retrieving Results for Index 70: ponta do sol.
Closest hotel is Escarpas do Lago.
------------
Retrieving Results for Index 88: cidreira.


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel is Novomar Pousada e Turismo Ltda.
------------
Retrieving Results for Index 138: morondava.
Closest hotel is La Case Bambou.
------------
Retrieving Results for Index 139: vaini.
Closest hotel is Dandeli Lake County.
------------
Retrieving Results for Index 176: nador.
Closest hotel is Hôtel La Giralda.
------------
Retrieving Results for Index 297: mogok.
Closest hotel is MOGOK HILL HOTEL.
------------
Retrieving Results for Index 444: birjand.
Closest hotel is تاکسی سرویس آرامش.
------------


In [6]:
print(json.dumps(response, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAFJOrYmtFXqU22mRRyVqKSmgieY6RkXZ1CnwkxZQsAitJiN0C-Ulym3UEdZ5uc1VOER1cbyIbT82i3CGoFW24WyZtJT0awf0zTOZgbo68ynq4UrXD0LxIF2RcyObvEw86m2PFP_0QQR-vRwRPhFVDCvu4wAGgpR7EJIDGMkVkqzf4eddvEBx2tIxQTvZe1YECymwUxta6UIN53fNHu66fEzujCTTr0EaIkvK8stmXpNu9A22LSOQvYUiVtrrwNIFZawb7OA5LN6jrXPcM2smwhrh8dGeNruVsWzyOpna3KWKRmZTQwb_sPBp0MeWfBMNqf4Q-c9OECPWyR7swoWgLqItOhDcPYgH9rnXz_GHZqV05lvoAgVcFx8QC_IYRFFHShIQfonDMptj-jQs-3ddGD4VExoUq-QlkIXf4sx1MQOhWMKnpFOMlcI",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.8625177,
                    "lng": 59.2037759
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.86386753029149,
                        "lng": 59.20508738029152
                    },
                    "southwest": {
                        "lat": 32.86116956970849

In [7]:
# NOTE: Do not change any of the code in this cell
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
SDR = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# NOTE: be sure to update with your DataFrame name
locations = hotel_df[["Lat", "Lng"]]
#coordinate 
coordinates = [
    (40.71, -74.00),
    (30.26, -97.74),
    (46.87, -96.78),
    (47.60, -122.33),
    (32.71, -117.16)
]

#Google Map layears
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=SDR
)


In [8]:
# Add marker layer ontop of heat map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))